In [6]:
import cv2
import pytesseract

def scale_in_meters(text):
    if(text=="1 km" or text=="1km"):
        dis=1000
    elif(text=="2 km" or text=="2km"):
        dis=2000
    elif(text=="5 km" or text=="5km"):
        dis=5000
    elif(text=="500m" or text=="500 m"):
        dis=500
    elif(text=="100m" or text=="100 m"):
        dis=100
    elif(text=="50m" or text=="50 m"):
        dis=50
    elif(text=="20m" or text=="20 m"):
        dis=20
    elif(text=="10m" or text=="10 m"):
        dis=10
    return dis



# Loading the image
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Adjust the path as needed

image_path = r"C:\Users\arpan\OneDrive\Pictures\Screenshots\Screenshot 2024-05-24 113857.png"
image = cv2.imread(image_path)

# Display the original image to get the coordinates for cropping
# This step is helpful for manually identifying the region
cv2.imshow('Original Image', image)
cv2.waitKey(0)

# Define the region of interest (ROI) for the scale 


height, width, _ = image.shape
x = width - 450  
y = height - 150  
w = 450  
h = 150 

# Crop the region containing the scale
roi = image[y:y+h, x:x+w]



extracted_text = pytesseract.image_to_string(roi)

import re

def extract_digit_with_following_chars(input_string):
    
    pattern = r'\d{1,3}.*?m'
    
    
    matches = re.findall(pattern, input_string)
    
    return matches


result = extract_digit_with_following_chars(extracted_text)
if len(result)==0:
    print("No scale")

print('Extracted Text:', result[-1])


# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply thresholding
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# Detect edges
edges = cv2.Canny(thresh, 50, 150, apertureSize=3)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# Filter and find the scale bar contour
scale_contour = None
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    if w > 50 and h < 20:  # Assuming the scale bar has specific width and height
        scale_contour = contour
        break

# Draw the detected contour (for visualization)
if scale_contour is not None:
    cv2.drawContours(image, [scale_contour], -1, (0, 255, 0), 2)
    x, y, w, h = cv2.boundingRect(scale_contour)
    scale_bar_length_pixels = w
    scale_bar_length_mm = scale_in_meters(result[-1])  # Assuming the scale bar is 1 km (1000 meters)
    
    # Calculate pixels per meter
    meter_per_pixel = scale_bar_length_mm / scale_bar_length_pixels
    
    print(f"Scale bar length: {scale_bar_length_pixels} pixels")
    print(f"meters per pixel: {meter_per_pixel} m/px")
else:
    print("Scale bar not detected")

# Display the result
cv2.imshow('Scale Bar Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()



Extracted Text: 5km
Scale bar length: 110 pixels
meters per pixel: 45.45454545454545 m/px
